<a href="https://colab.research.google.com/github/Raph3103/hw2AI/blob/main/Low_Collateral_Blockchain_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Collateral Blockchain Project


In [564]:
pip install pandas

In [565]:
import pandas as pd
import pickle

Get the stat file: We will start by obtaining the stat file, which contains relevant data for our analysis.

In [566]:
data = pd.read_pickle('/content/HistoryPoolBorrowersStats.pkl')

df = pd.DataFrame.from_dict(data,orient='index')

chemin_fichier_csv = 'newStats.csv'

df.to_csv(chemin_fichier_csv, index=True)


Add columns and modify the existing file: We will enhance the stat file by adding new columns and modifying existing ones to better suit our analysis requirements.


In [567]:
# Read the CSV file into a dataframe
df = pd.read_csv('newStats.csv')

# Remove columns
columns_to_remove = ['datesSupplyCollateral', 'datesLoan','datesReimbursement','datesWithdrawCollateral','timeUTCFirstAnyTransactionAccount','borrowerAgeInYears','totalLoans','NumLoans','MeanLoans','totalTimeloans','MeanTimeLoans','totalSupplyCollateral','NumSupplyCollateral','MeanSupplyCollateral','totalReimbursements','NumReimbursements','MeanReimbursements','NumTransactions']  # Specify the names of the columns to remove
df = df.drop(columns=columns_to_remove)


# Save the modified dataframe to a new CSV file
df.to_csv('newStats.csv', index=False)

This part transfor the dict from date keys to tomestamps keys and also add a new value named ratio for each timestamsp if size debt = 0 then collateralDebtRatio = infinity

In [568]:
import pandas as pd
import json
from datetime import datetime
import time

# Load the csv file
df = pd.read_csv('newStats.csv')


# Define a helper function to convert a date to a timestamp
def date_to_timestamp(date_str):
    dt_object = datetime.strptime(date_str, "%Y-%m-%d")
    timestamp = int(time.mktime(dt_object.timetuple()))
    return timestamp

# Iterate over the Calendar column
for i, cell in enumerate(df['Calendar']):
    if isinstance(cell, str):
        # Replace single quotes with double quotes
        cell = cell.replace("'", '"')
        # Load the dictionary
        calendar_dict = json.loads(cell)
        new_calendar_dict = {}

        # Iterate over the days in the calendar_dict
        for date_str, day_dict in calendar_dict.items():
            # Convert the date to a timestamp and replace it in the dictionary
            timestamp = date_to_timestamp(date_str)

            # Compute the ratio and insert it into the dictionary
            if day_dict['sizeDebtUSD'] != 0:
                ratio = day_dict['sizeCollateralUSD'] / day_dict['sizeDebtUSD']
            else:
                ratio = float('inf')
            day_dict['collateralDebtRatio'] = ratio

            new_calendar_dict[timestamp] = day_dict

        # Replace the row's value in the DataFrame with the updated dictionary
        df.at[i, 'Calendar'] = json.dumps(new_calendar_dict)
    else:
        continue

# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)


adding most frequent time in the day

In [569]:
import pandas as pd
from datetime import datetime

# Define function to convert UNIX timestamp to time of day category
def timestamp_to_category(timestamp):
    # Convert timestamp to datetime
    dt_object = datetime.fromtimestamp(int(float(timestamp)))  # Modify this line
    hour = dt_object.hour

    # Assign time of day category based on hour
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

# Define function to find the most frequent category in a list
def most_frequent_category(lst):
    return max(set(lst), key=lst.count) if lst else None

# Load the csv file
df = pd.read_csv('newStats.csv')

# Define the timestamp columns to examine
timestamp_cols = ['timeStampsSupplyCollateral', 'timeStampsLoans', 'timeStampsReimbursement', 'timeStampsWithdrawCollateral', 'timeStampFirstAnyTransactionAccount']

# Create a new column for the time of day of most transactions
df['MostFrequentTransactionTimeOfDay'] = ""

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    categories = []
    # For each timestamp column, convert the timestamps to categories and add them to the list
    for col in timestamp_cols:
        if pd.isnull(row[col]):
            continue
        timestamps = str(row[col]).split()
        for timestamp in timestamps:
            categories.append(timestamp_to_category(timestamp))
    # Determine the most frequent category and assign it to the new column
    df.at[i, 'MostFrequentTransactionTimeOfDay'] = most_frequent_category(categories)

# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)


count the number of days with high ratio(1 but we can change the value if needed) for each user, using the new value in the calendar colomn

In [570]:
import json

threshold = 2.2

def count_days_with_high_ratio(calendar_str):
    # Check if calendar_str is a string
    if not isinstance(calendar_str, str):
        # Return None (or 0, depending on your needs) if not
        return None  # or return 0

    # Convert string to dictionary
    calendar = json.loads(calendar_str)

    count = 0
    for day, attributes in calendar.items():
        if 'collateralDebtRatio' in attributes and attributes['collateralDebtRatio'] > threshold:
            count += 1
    return count

# Create a new column for the count of days with collateralDebtRatio > 13
df['DaysWithHighRatio'] = df['Calendar'].apply(count_days_with_high_ratio)
# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)

if the user paid x percent in the last y month the val is true

In [571]:
import json
import time

x_percent=0.1

y_month=9
def loan_repaid_x_percent_in_last_y_months(calendar):
    # Define the timestamp for 9 months before the day you stopped computing the file
    six_months_ago = 1668297600 - y_month*30*24*60*60

    # Load the dictionary from the calendar cell
    if isinstance(calendar, str):
        calendar_dict = json.loads(calendar)
    else:
        return False

    # Initialize the total loan and total repayment amounts
    total_loan = 0
    total_repayment = 0

    # Iterate over the days in the calendar_dict
    for timestamp, day_dict in calendar_dict.items():
        # Convert the timestamp to an integer
        timestamp = int(timestamp)

        # If the timestamp is within the last 6 months and not before the user's first day
        if six_months_ago <= timestamp <= 1668297600:
            # Add the loan and repayment amounts for the day to the totals
            total_loan += day_dict['sizeDebtUSD']
            total_repayment += day_dict['amountPaymentsOnDay']

    # If the total loan amount is 0, return False
    if total_loan == 0:
        return False

    # Calculate the repayment percentage
    repayment_percentage = total_repayment / total_loan

    # If the repayment percentage is at least 66%, return True
    if repayment_percentage >= x_percent:
        return True
    else:
        return False

# Apply the function to the Calendar column to create a new column
df['LoanRepaidxPercentInLastyMonths'] = df['Calendar'].apply(loan_repaid_x_percent_in_last_y_months)


# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)

Create a function to evaluate user behavior: We will develop a function that determines whether a user is classified as good or bad based on their activities on our platform. This function will help us assess user behavior and identify potential risks.


In [572]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

six_months_ago = today - timedelta(days=15*30)  # Assuming each month has 30 days

# Convert the dates to timestamps
timestamp_today = int(today.timestamp())
timestamp_six_months_ago = int(six_months_ago.timestamp())


def evaluate_user_behavior(row):
    conditions_met = 0


    if row['DaysWithHighRatio'] > 60:
        conditions_met += 2
        if row['LoanRepaidxPercentInLastyMonths']:
          conditions_met += 4
    if row['timeStampFirstAnyTransactionAccount'] < timestamp_six_months_ago :
        conditions_met += 1

    if conditions_met==5 or conditions_met == 6 or conditions_met==7 or conditions_met == 3:
        return 'Good'
    elif conditions_met ==0 :
        return 'Bad'
    else:
        return 'Unknown'


Run the function for all rows: We will iterate over each row of the stat file and apply the user evaluation function to determine the classification for each user.


In [573]:
df['user_class'] = df.apply(evaluate_user_behavior, axis=1)

Add the new column to the CSV: After evaluating all the users, we will add the newly generated classification column to the CSV file.


In [574]:
import pandas as pd

df = df.drop(columns='Calendar')

# Save the DataFrame as a CSV file
df.to_csv('updated_data.csv', index=False)

Run a machine learning model: With the updated CSV file, we will utilize a machine learning model to gain insights and predictions based on the user behavior data.


In [575]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np

# Read the data
df = pd.read_csv('updated_data.csv')
#print(df.describe())
# Drop unnecessary column
df = df.drop(columns=['Unnamed: 0'])
#df = df.drop(columns=['Calendar'])
# Handle timestamp strings
# Handle timestamp strings
for col in ['timeStampsSupplyCollateral', 'timeStampsLoans', 'timeStampsReimbursement', 'timeStampsWithdrawCollateral','maximumDebt', 'atTimeMaximumDebtCollateralProvided']:
    df= df.drop(columns=col)

rows_with_nan = df[df.isna().any(axis=1)]

df.dropna()
# Convert boolean to int
df['LoanRepaidxPercentInLastyMonths'] = df['LoanRepaidxPercentInLastyMonths'].astype(int)

# Handle categorical columns
le = preprocessing.LabelEncoder()
df['MostFrequentTransactionTimeOfDay'] = le.fit_transform(df['MostFrequentTransactionTimeOfDay'])
df['user_class'] = le.fit_transform(df['user_class'])

# Extract features from Calendar dictionary (you need to specify this according to the dictionary's structure)
print(df.describe())
# Normalize numerical columns
scaler = StandardScaler()
num_cols = ['sizeLoansUSD', 'sizeCollateralUSD', 'sizeReimbursementsUSD', 'numLoansUser', 'ratioCollateralToLoans', 'averageOfDailyCollateralToDebt', 'numTransactionsUser', 'DaysWithHighRatio']
df[num_cols] = scaler.fit_transform(df[num_cols])

# Define features and target
for cols in ['DaysWithHighRatio','timeStampFirstAnyTransactionAccount','LoanRepaidxPercentInLastyMonths']:
  df = df.drop(columns=cols)
X = df.drop('user_class', axis=1)
y = df['user_class']

#print(X.head())
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)


# Drop rows with NaN values
X_train = X_train.dropna()
y_train = y_train[X_train.index]

# Create a model
rfc = RandomForestClassifier( n_estimators=100, random_state=42)

# Fit the model
rfc.fit(X_train, y_train)

# Make predictions
y_pred = rfc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(y_test)
print('-------------------------------------')
print(y_pred)



       sizeLoansUSD  sizeCollateralUSD  sizeReimbursementsUSD  numLoansUser  \
count  1.900000e+02       1.900000e+02           1.900000e+02    190.000000   
mean   8.134692e+05       1.542747e+06           6.428475e+05      5.000000   
std    3.418124e+06       6.631302e+06           3.151741e+06     13.317054   
min    1.000000e+02       1.101659e+02           4.612872e+00      1.000000   
25%    7.625024e+02       1.296070e+03           4.954414e+02      1.000000   
50%    1.442505e+04       2.721747e+04           9.930242e+03      2.000000   
75%    1.257161e+05       2.439932e+05           1.005268e+05      4.000000   
max    3.913494e+07       6.396438e+07           3.914135e+07    156.000000   

       numReimbursementsUser  ratioCollateralToLoans  \
count             190.000000              190.000000   
mean                3.200000                2.049878   
std                 7.024673                1.284903   
min                 1.000000                0.135575   
25%     

Analyze performance: Finally, we will analyze the performance of our machine learning model and evaluate its effectiveness in predicting user behavior accurately.

In [576]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

# Make predictions on the test set
y_pred = rfc.predict(X_test)

# Print the Confusion Matrix and slice it into four pieces for future use
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', cm)

# Calculate Accuracy, Precision, Recall, and F1-Score
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}\n')
print(f'Classification Report:\n{report}')

# Calculate Area Under the Receiver Operating Characteristic Curve (ROC AUC)
roc_auc = roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1])
print('Area Under ROC Curve: ', roc_auc)


Confusion Matrix:
 [[  0   0   9]
 [  0   1  17]
 [  3   8 115]]
Accuracy: 0.7581699346405228

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.11      0.06      0.07        18
           2       0.82      0.91      0.86       126

    accuracy                           0.76       153
   macro avg       0.31      0.32      0.31       153
weighted avg       0.68      0.76      0.72       153



ValueError: ignored